In [1]:
import pandas as pd
import numpy as np
import re
import sklearn
from sklearn import feature_extraction, tree, model_selection, metrics
import whois as wh
import json
from flare.data_science.features import domain_tld_extract
from flare.data_science.features import domain_extract

3. Cargue la información del archivo large_even.json en una lista, muestre la cantidad de
registros total (deben ser 746, 909). Este es nuestro tráfico inicial!


In [2]:
df = [json.loads(i) for i in open('./large_eve.json','r', encoding='utf-8')]

In [3]:
for i in range(5):
  print(df[i])

{'timestamp': '2017-07-22T17:33:15.784100-0500', 'flow_id': 283570730437312, 'pcap_cnt': 18519, 'event_type': 'alert', 'vlan': 130, 'src_ip': '192.168.203.200', 'src_port': 2328, 'dest_ip': '192.181.145.109', 'dest_port': 445, 'proto': 'TCP', 'alert': {'action': 'allowed', 'gid': 1, 'signature_id': 2001569, 'rev': 15, 'signature': 'ET SCAN Behavioral Unusual Port 445 traffic Potential Scan or Infection', 'category': 'Misc activity', 'severity': 3}}
{'timestamp': '2017-07-22T17:33:16.571273-0500', 'flow_id': 1519048138171115, 'pcap_cnt': 21531, 'event_type': 'ssh', 'vlan': 140, 'src_ip': '192.168.204.60', 'src_port': 62990, 'dest_ip': '192.168.26.254', 'dest_port': 22, 'proto': 'TCP', 'ssh': {'client': {'proto_version': '2.0', 'software_version': 'OpenSSH_5.0'}, 'server': {'proto_version': '1.99', 'software_version': 'Cisco-1.25'}}}
{'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18

7. Debido a que la data consiste en json anidados, utilice la característica json_normalize para
normalizar la información y asignarla en un dataframe.

In [4]:
data = pd.json_normalize(df)

In [11]:
pd.set_option('display.max_columns', None)
data.head()


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,alert.action,alert.gid,alert.signature_id,alert.rev,alert.signature,alert.category,alert.severity,ssh.client.proto_version,ssh.client.software_version,ssh.server.proto_version,ssh.server.software_version,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,tx_id,http.hostname,http.url,http.http_user_agent,http.http_content_type,http.http_method,http.protocol,http.status,http.redirect,http.length,app_proto,fileinfo.filename,fileinfo.state,fileinfo.md5,fileinfo.stored,fileinfo.size,fileinfo.tx_id,http.http_refer,dns.rcode,dns.ttl,tls.session_resumed,tls.version,vars.flowints.http.anomaly.count,tls.subject,tls.issuerdn,tls.serial,tls.fingerprint,tls.notbefore,tls.notafter,flow.pkts_toserver,flow.pkts_toclient,flow.bytes_toserver,flow.bytes_toclient,flow.start,flow.end,flow.age,flow.state,flow.reason,flow.alerted,icmp_type,icmp_code,tcp.tcp_flags,tcp.tcp_flags_ts,tcp.tcp_flags_tc,tcp.syn,tcp.rst,tcp.ack,tcp.state,smtp.helo,smtp.mail_from,smtp.rcpt_to,email.status,vars.flowints.smtp.anomaly.count,http.xff,tcp.fin,tcp.psh,app_proto_tc,dns.rdata,app_proto_ts,vars.flowints.tls.anomaly.count,stats.uptime,stats.decoder.pkts,stats.decoder.bytes,stats.decoder.invalid,stats.decoder.ipv4,stats.decoder.ipv6,stats.decoder.ethernet,stats.decoder.raw,stats.decoder.null,stats.decoder.sll,stats.decoder.tcp,stats.decoder.udp,stats.decoder.sctp,stats.decoder.icmpv4,stats.decoder.icmpv6,stats.decoder.ppp,stats.decoder.pppoe,stats.decoder.gre,stats.decoder.vlan,stats.decoder.vlan_qinq,stats.decoder.teredo,stats.decoder.ipv4_in_ipv6,stats.decoder.ipv6_in_ipv6,stats.decoder.mpls,stats.decoder.avg_pkt_size,stats.decoder.max_pkt_size,stats.decoder.erspan,stats.decoder.ipraw.invalid_ip_version,stats.decoder.ltnull.pkt_too_small,stats.decoder.ltnull.unsupported_type,stats.decoder.dce.pkt_too_small,stats.flow.memcap,stats.flow.tcp,stats.flow.udp,stats.flow.icmpv4,stats.flow.icmpv6,stats.flow.spare,stats.flow.emerg_mode_entered,stats.flow.emerg_mode_over,stats.flow.tcp_reuse,stats.flow.memuse,stats.defrag.ipv4.fragments,stats.defrag.ipv4.reassembled,stats.defrag.ipv4.timeouts,stats.defrag.ipv6.fragments,stats.defrag.ipv6.reassembled,stats.defrag.ipv6.timeouts,stats.defrag.max_frag_hits,stats.tcp.sessions,stats.tcp.ssn_memcap_drop,stats.tcp.pseudo,stats.tcp.pseudo_failed,stats.tcp.invalid_checksum,stats.tcp.no_flow,stats.tcp.syn,stats.tcp.synack,stats.tcp.rst,stats.tcp.segment_memcap_drop,stats.tcp.stream_depth_reached,stats.tcp.reassembly_gap,stats.tcp.overlap,stats.tcp.overlap_diff_data,stats.tcp.insert_data_normal_fail,stats.tcp.insert_data_overlap_fail,stats.tcp.insert_list_fail,stats.tcp.memuse,stats.tcp.reassembly_memuse,stats.detect.alert,stats.app_layer.flow.http,stats.app_layer.flow.ftp,stats.app_layer.flow.smtp,stats.app_layer.flow.tls,stats.app_layer.flow.ssh,stats.app_layer.flow.imap,stats.app_layer.flow.msn,stats.app_layer.flow.smb,stats.app_layer.flow.dcerpc_tcp,stats.app_layer.flow.dns_tcp,stats.app_layer.flow.failed_tcp,stats.app_layer.flow.dcerpc_udp,stats.app_layer.flow.dns_udp,stats.app_layer.flow.failed_udp,stats.app_layer.tx.http,stats.app_layer.tx.ftp,stats.app_layer.tx.smtp,stats.app_layer.tx.tls,stats.app_layer.tx.ssh,stats.app_layer.tx.smb,stats.app_layer.tx.dcerpc_tcp,stats.app_layer.tx.dns_tcp,stats.app_layer.tx.dns_udp,stats.flow_mgr.closed_pruned,stats.flow_mgr.new_pruned,stats.flow_mgr.est_pruned,stats.flow_mgr.bypassed_pruned,stats.flow_mgr.flows_checked,stats.flow_mgr.flows_notimeout,stats.flow_mgr.flows_timeout,stats.flow_mgr.flows_timeout_inuse,stats.flow_mgr.flows_removed,stats.flow_mgr.rows_checked,stats.flow_mgr.rows_skipped,stats.flow_mgr.rows_empty,stats.flow_mgr.rows_busy,stats.flow_mgr.rows_maxlen,stats.file_store.open_files,stats.dns.memuse,stats.dns.memcap_state,stats.dns.memcap_global,stats.http.memuse,stats.http.memcap,tcp.ecn,tcp.cwr
0,2017-07-22T17:33:15.784100-0500,2.835707e+14,18519.0,alert,130.0,192.168.203.200,2328.0,192.181.145.10

4. Debido a que estamos buscando dominios DGA, del total de registros, solamente estamos
interesados en los registros DNS. Cargue únicamente aquellos registros en cuya llave se
encuentra “DNS”.


In [5]:
data = data.loc[data['event_type'] == 'dns']

5. Muestre la nueva cantidad de registros filtrados. Deben ser 21484. Esta es una cantidad
mucho más manejable, pero aún se debe seguir depurando la información a buscar

In [11]:
len(data)

15749

6. Muestre la información de 2 registros cualesquiera.


In [33]:
data.head(5)

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,alert.action,alert.gid,alert.signature_id,alert.rev,alert.signature,alert.category,alert.severity,ssh.client.proto_version,ssh.client.software_version,ssh.server.proto_version,ssh.server.software_version,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,tx_id,http.hostname,http.url,http.http_user_agent,http.http_content_type,http.http_method,http.protocol,http.status,http.redirect,http.length,app_proto,fileinfo.filename,fileinfo.state,fileinfo.md5,fileinfo.stored,fileinfo.size,fileinfo.tx_id,http.http_refer,dns.rcode,dns.ttl,tls.session_resumed,tls.version,vars.flowints.http.anomaly.count,tls.subject,tls.issuerdn,tls.serial,tls.fingerprint,tls.notbefore,tls.notafter,flow.pkts_toserver,flow.pkts_toclient,flow.bytes_toserver,flow.bytes_toclient,flow.start,flow.end,flow.age,flow.state,flow.reason,flow.alerted,icmp_type,icmp_code,tcp.tcp_flags,tcp.tcp_flags_ts,tcp.tcp_flags_tc,tcp.syn,tcp.rst,tcp.ack,tcp.state,smtp.helo,smtp.mail_from,smtp.rcpt_to,email.status,vars.flowints.smtp.anomaly.count,http.xff,tcp.fin,tcp.psh,app_proto_tc,dns.rdata,app_proto_ts,vars.flowints.tls.anomaly.count,stats.uptime,stats.decoder.pkts,stats.decoder.bytes,stats.decoder.invalid,stats.decoder.ipv4,stats.decoder.ipv6,stats.decoder.ethernet,stats.decoder.raw,stats.decoder.null,stats.decoder.sll,stats.decoder.tcp,stats.decoder.udp,stats.decoder.sctp,stats.decoder.icmpv4,stats.decoder.icmpv6,stats.decoder.ppp,stats.decoder.pppoe,stats.decoder.gre,stats.decoder.vlan,stats.decoder.vlan_qinq,stats.decoder.teredo,stats.decoder.ipv4_in_ipv6,stats.decoder.ipv6_in_ipv6,stats.decoder.mpls,stats.decoder.avg_pkt_size,stats.decoder.max_pkt_size,stats.decoder.erspan,stats.decoder.ipraw.invalid_ip_version,stats.decoder.ltnull.pkt_too_small,stats.decoder.ltnull.unsupported_type,stats.decoder.dce.pkt_too_small,stats.flow.memcap,stats.flow.tcp,stats.flow.udp,stats.flow.icmpv4,stats.flow.icmpv6,stats.flow.spare,stats.flow.emerg_mode_entered,stats.flow.emerg_mode_over,stats.flow.tcp_reuse,stats.flow.memuse,stats.defrag.ipv4.fragments,stats.defrag.ipv4.reassembled,stats.defrag.ipv4.timeouts,stats.defrag.ipv6.fragments,stats.defrag.ipv6.reassembled,stats.defrag.ipv6.timeouts,stats.defrag.max_frag_hits,stats.tcp.sessions,stats.tcp.ssn_memcap_drop,stats.tcp.pseudo,stats.tcp.pseudo_failed,stats.tcp.invalid_checksum,stats.tcp.no_flow,stats.tcp.syn,stats.tcp.synack,stats.tcp.rst,stats.tcp.segment_memcap_drop,stats.tcp.stream_depth_reached,stats.tcp.reassembly_gap,stats.tcp.overlap,stats.tcp.overlap_diff_data,stats.tcp.insert_data_normal_fail,stats.tcp.insert_data_overlap_fail,stats.tcp.insert_list_fail,stats.tcp.memuse,stats.tcp.reassembly_memuse,stats.detect.alert,stats.app_layer.flow.http,stats.app_layer.flow.ftp,stats.app_layer.flow.smtp,stats.app_layer.flow.tls,stats.app_layer.flow.ssh,stats.app_layer.flow.imap,stats.app_layer.flow.msn,stats.app_layer.flow.smb,stats.app_layer.flow.dcerpc_tcp,stats.app_layer.flow.dns_tcp,stats.app_layer.flow.failed_tcp,stats.app_layer.flow.dcerpc_udp,stats.app_layer.flow.dns_udp,stats.app_layer.flow.failed_udp,stats.app_layer.tx.http,stats.app_layer.tx.ftp,stats.app_layer.tx.smtp,stats.app_layer.tx.tls,stats.app_layer.tx.ssh,stats.app_layer.tx.smb,stats.app_layer.tx.dcerpc_tcp,stats.app_layer.tx.dns_tcp,stats.app_layer.tx.dns_udp,stats.flow_mgr.closed_pruned,stats.flow_mgr.new_pruned,stats.flow_mgr.est_pruned,stats.flow_mgr.bypassed_pruned,stats.flow_mgr.flows_checked,stats.flow_mgr.flows_notimeout,stats.flow_mgr.flows_timeout,stats.flow_mgr.flows_timeout_inuse,stats.flow_mgr.flows_removed,stats.flow_mgr.rows_checked,stats.flow_mgr.rows_skipped,stats.flow_mgr.rows_empty,stats.flow_mgr.rows_busy,stats.flow_mgr.rows_maxlen,stats.file_store.open_files,stats.dns.memuse,stats.dns.memcap_state,stats.dns.memcap_global,stats.http.memuse,stats.http.memcap,tcp.ecn,tcp.cwr
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863

In [10]:
data.tail()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,alert.action,alert.gid,alert.signature_id,alert.rev,alert.signature,alert.category,alert.severity,ssh.client.proto_version,ssh.client.software_version,ssh.server.proto_version,ssh.server.software_version,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,tx_id,http.hostname,http.url,http.http_user_agent,http.http_content_type,http.http_method,http.protocol,http.status,http.redirect,http.length,app_proto,fileinfo.filename,fileinfo.state,fileinfo.md5,fileinfo.stored,fileinfo.size,fileinfo.tx_id,http.http_refer,dns.rcode,dns.ttl,tls.session_resumed,tls.version,vars.flowints.http.anomaly.count,tls.subject,tls.issuerdn,tls.serial,tls.fingerprint,tls.notbefore,tls.notafter,flow.pkts_toserver,flow.pkts_toclient,flow.bytes_toserver,flow.bytes_toclient,flow.start,flow.end,flow.age,flow.state,flow.reason,flow.alerted,icmp_type,icmp_code,tcp.tcp_flags,tcp.tcp_flags_ts,tcp.tcp_flags_tc,tcp.syn,tcp.rst,tcp.ack,tcp.state,smtp.helo,smtp.mail_from,smtp.rcpt_to,email.status,vars.flowints.smtp.anomaly.count,http.xff,tcp.fin,tcp.psh,app_proto_tc,dns.rdata,app_proto_ts,vars.flowints.tls.anomaly.count,stats.uptime,stats.decoder.pkts,stats.decoder.bytes,stats.decoder.invalid,stats.decoder.ipv4,stats.decoder.ipv6,stats.decoder.ethernet,stats.decoder.raw,stats.decoder.null,stats.decoder.sll,stats.decoder.tcp,stats.decoder.udp,stats.decoder.sctp,stats.decoder.icmpv4,stats.decoder.icmpv6,stats.decoder.ppp,stats.decoder.pppoe,stats.decoder.gre,stats.decoder.vlan,stats.decoder.vlan_qinq,stats.decoder.teredo,stats.decoder.ipv4_in_ipv6,stats.decoder.ipv6_in_ipv6,stats.decoder.mpls,stats.decoder.avg_pkt_size,stats.decoder.max_pkt_size,stats.decoder.erspan,stats.decoder.ipraw.invalid_ip_version,stats.decoder.ltnull.pkt_too_small,stats.decoder.ltnull.unsupported_type,stats.decoder.dce.pkt_too_small,stats.flow.memcap,stats.flow.tcp,stats.flow.udp,stats.flow.icmpv4,stats.flow.icmpv6,stats.flow.spare,stats.flow.emerg_mode_entered,stats.flow.emerg_mode_over,stats.flow.tcp_reuse,stats.flow.memuse,stats.defrag.ipv4.fragments,stats.defrag.ipv4.reassembled,stats.defrag.ipv4.timeouts,stats.defrag.ipv6.fragments,stats.defrag.ipv6.reassembled,stats.defrag.ipv6.timeouts,stats.defrag.max_frag_hits,stats.tcp.sessions,stats.tcp.ssn_memcap_drop,stats.tcp.pseudo,stats.tcp.pseudo_failed,stats.tcp.invalid_checksum,stats.tcp.no_flow,stats.tcp.syn,stats.tcp.synack,stats.tcp.rst,stats.tcp.segment_memcap_drop,stats.tcp.stream_depth_reached,stats.tcp.reassembly_gap,stats.tcp.overlap,stats.tcp.overlap_diff_data,stats.tcp.insert_data_normal_fail,stats.tcp.insert_data_overlap_fail,stats.tcp.insert_list_fail,stats.tcp.memuse,stats.tcp.reassembly_memuse,stats.detect.alert,stats.app_layer.flow.http,stats.app_layer.flow.ftp,stats.app_layer.flow.smtp,stats.app_layer.flow.tls,stats.app_layer.flow.ssh,stats.app_layer.flow.imap,stats.app_layer.flow.msn,stats.app_layer.flow.smb,stats.app_layer.flow.dcerpc_tcp,stats.app_layer.flow.dns_tcp,stats.app_layer.flow.failed_tcp,stats.app_layer.flow.dcerpc_udp,stats.app_layer.flow.dns_udp,stats.app_layer.flow.failed_udp,stats.app_layer.tx.http,stats.app_layer.tx.ftp,stats.app_layer.tx.smtp,stats.app_layer.tx.tls,stats.app_layer.tx.ssh,stats.app_layer.tx.smb,stats.app_layer.tx.dcerpc_tcp,stats.app_layer.tx.dns_tcp,stats.app_layer.tx.dns_udp,stats.flow_mgr.closed_pruned,stats.flow_mgr.new_pruned,stats.flow_mgr.est_pruned,stats.flow_mgr.bypassed_pruned,stats.flow_mgr.flows_checked,stats.flow_mgr.flows_notimeout,stats.flow_mgr.flows_timeout,stats.flow_mgr.flows_timeout_inuse,stats.flow_mgr.flows_removed,stats.flow_mgr.rows_checked,stats.flow_mgr.rows_skipped,stats.flow_mgr.rows_empty,stats.flow_mgr.rows_busy,stats.flow_mgr.rows_maxlen,stats.file_store.open_files,stats.dns.memuse,stats.dns.memcap_state,stats.dns.memcap_global,stats.http.memuse,stats.http.memcap,tcp.ecn,tcp.cwr
716752,2017-07-22T19:47:03.373435-0500,8.918761e+14,4441474.0,dns,120.0,192.168.207.4,53.0,192.168.202.1

8. Como estamos buscando dominios DGA, debemos filtrar los registros DNS para aquellos
registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio), se debió
bajar la cantidad a 2849 registros.

In [6]:
data2 = data.loc[data['dns.rrtype'] == 'A']

In [13]:
len(data2)

2849

In [14]:
pd.set_option('display.max_columns', None)
data2.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,alert.action,alert.gid,alert.signature_id,alert.rev,alert.signature,alert.category,alert.severity,ssh.client.proto_version,ssh.client.software_version,ssh.server.proto_version,ssh.server.software_version,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,tx_id,http.hostname,http.url,http.http_user_agent,http.http_content_type,http.http_method,http.protocol,http.status,http.redirect,http.length,app_proto,fileinfo.filename,fileinfo.state,fileinfo.md5,fileinfo.stored,fileinfo.size,fileinfo.tx_id,http.http_refer,dns.rcode,dns.ttl,tls.session_resumed,tls.version,vars.flowints.http.anomaly.count,tls.subject,tls.issuerdn,tls.serial,tls.fingerprint,tls.notbefore,tls.notafter,flow.pkts_toserver,flow.pkts_toclient,flow.bytes_toserver,flow.bytes_toclient,flow.start,flow.end,flow.age,flow.state,flow.reason,flow.alerted,icmp_type,icmp_code,tcp.tcp_flags,tcp.tcp_flags_ts,tcp.tcp_flags_tc,tcp.syn,tcp.rst,tcp.ack,tcp.state,smtp.helo,smtp.mail_from,smtp.rcpt_to,email.status,vars.flowints.smtp.anomaly.count,http.xff,tcp.fin,tcp.psh,app_proto_tc,dns.rdata,app_proto_ts,vars.flowints.tls.anomaly.count,stats.uptime,stats.decoder.pkts,stats.decoder.bytes,stats.decoder.invalid,stats.decoder.ipv4,stats.decoder.ipv6,stats.decoder.ethernet,stats.decoder.raw,stats.decoder.null,stats.decoder.sll,stats.decoder.tcp,stats.decoder.udp,stats.decoder.sctp,stats.decoder.icmpv4,stats.decoder.icmpv6,stats.decoder.ppp,stats.decoder.pppoe,stats.decoder.gre,stats.decoder.vlan,stats.decoder.vlan_qinq,stats.decoder.teredo,stats.decoder.ipv4_in_ipv6,stats.decoder.ipv6_in_ipv6,stats.decoder.mpls,stats.decoder.avg_pkt_size,stats.decoder.max_pkt_size,stats.decoder.erspan,stats.decoder.ipraw.invalid_ip_version,stats.decoder.ltnull.pkt_too_small,stats.decoder.ltnull.unsupported_type,stats.decoder.dce.pkt_too_small,stats.flow.memcap,stats.flow.tcp,stats.flow.udp,stats.flow.icmpv4,stats.flow.icmpv6,stats.flow.spare,stats.flow.emerg_mode_entered,stats.flow.emerg_mode_over,stats.flow.tcp_reuse,stats.flow.memuse,stats.defrag.ipv4.fragments,stats.defrag.ipv4.reassembled,stats.defrag.ipv4.timeouts,stats.defrag.ipv6.fragments,stats.defrag.ipv6.reassembled,stats.defrag.ipv6.timeouts,stats.defrag.max_frag_hits,stats.tcp.sessions,stats.tcp.ssn_memcap_drop,stats.tcp.pseudo,stats.tcp.pseudo_failed,stats.tcp.invalid_checksum,stats.tcp.no_flow,stats.tcp.syn,stats.tcp.synack,stats.tcp.rst,stats.tcp.segment_memcap_drop,stats.tcp.stream_depth_reached,stats.tcp.reassembly_gap,stats.tcp.overlap,stats.tcp.overlap_diff_data,stats.tcp.insert_data_normal_fail,stats.tcp.insert_data_overlap_fail,stats.tcp.insert_list_fail,stats.tcp.memuse,stats.tcp.reassembly_memuse,stats.detect.alert,stats.app_layer.flow.http,stats.app_layer.flow.ftp,stats.app_layer.flow.smtp,stats.app_layer.flow.tls,stats.app_layer.flow.ssh,stats.app_layer.flow.imap,stats.app_layer.flow.msn,stats.app_layer.flow.smb,stats.app_layer.flow.dcerpc_tcp,stats.app_layer.flow.dns_tcp,stats.app_layer.flow.failed_tcp,stats.app_layer.flow.dcerpc_udp,stats.app_layer.flow.dns_udp,stats.app_layer.flow.failed_udp,stats.app_layer.tx.http,stats.app_layer.tx.ftp,stats.app_layer.tx.smtp,stats.app_layer.tx.tls,stats.app_layer.tx.ssh,stats.app_layer.tx.smb,stats.app_layer.tx.dcerpc_tcp,stats.app_layer.tx.dns_tcp,stats.app_layer.tx.dns_udp,stats.flow_mgr.closed_pruned,stats.flow_mgr.new_pruned,stats.flow_mgr.est_pruned,stats.flow_mgr.bypassed_pruned,stats.flow_mgr.flows_checked,stats.flow_mgr.flows_notimeout,stats.flow_mgr.flows_timeout,stats.flow_mgr.flows_timeout_inuse,stats.flow_mgr.flows_removed,stats.flow_mgr.rows_checked,stats.flow_mgr.rows_skipped,stats.flow_mgr.rows_empty,stats.flow_mgr.rows_busy,stats.flow_mgr.rows_maxlen,stats.file_store.open_files,stats.dns.memuse,stats.dns.memcap_state,stats.dns.memcap_global,stats.http.memuse,stats.http.memcap,tcp.ecn,tcp.cwr
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863

9. Filtre los dominios únicos y asígnelos a un dataframe. Debe obtener 177 registros únicos

In [7]:
data3= (data2.drop_duplicates(subset = "dns.rrname"))

In [16]:
len(data3)

177

In [17]:
pd.set_option('display.max_columns', None)
data3.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,alert.action,alert.gid,alert.signature_id,alert.rev,alert.signature,alert.category,alert.severity,ssh.client.proto_version,ssh.client.software_version,ssh.server.proto_version,ssh.server.software_version,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,tx_id,http.hostname,http.url,http.http_user_agent,http.http_content_type,http.http_method,http.protocol,http.status,http.redirect,http.length,app_proto,fileinfo.filename,fileinfo.state,fileinfo.md5,fileinfo.stored,fileinfo.size,fileinfo.tx_id,http.http_refer,dns.rcode,dns.ttl,tls.session_resumed,tls.version,vars.flowints.http.anomaly.count,tls.subject,tls.issuerdn,tls.serial,tls.fingerprint,tls.notbefore,tls.notafter,flow.pkts_toserver,flow.pkts_toclient,flow.bytes_toserver,flow.bytes_toclient,flow.start,flow.end,flow.age,flow.state,flow.reason,flow.alerted,icmp_type,icmp_code,tcp.tcp_flags,tcp.tcp_flags_ts,tcp.tcp_flags_tc,tcp.syn,tcp.rst,tcp.ack,tcp.state,smtp.helo,smtp.mail_from,smtp.rcpt_to,email.status,vars.flowints.smtp.anomaly.count,http.xff,tcp.fin,tcp.psh,app_proto_tc,dns.rdata,app_proto_ts,vars.flowints.tls.anomaly.count,stats.uptime,stats.decoder.pkts,stats.decoder.bytes,stats.decoder.invalid,stats.decoder.ipv4,stats.decoder.ipv6,stats.decoder.ethernet,stats.decoder.raw,stats.decoder.null,stats.decoder.sll,stats.decoder.tcp,stats.decoder.udp,stats.decoder.sctp,stats.decoder.icmpv4,stats.decoder.icmpv6,stats.decoder.ppp,stats.decoder.pppoe,stats.decoder.gre,stats.decoder.vlan,stats.decoder.vlan_qinq,stats.decoder.teredo,stats.decoder.ipv4_in_ipv6,stats.decoder.ipv6_in_ipv6,stats.decoder.mpls,stats.decoder.avg_pkt_size,stats.decoder.max_pkt_size,stats.decoder.erspan,stats.decoder.ipraw.invalid_ip_version,stats.decoder.ltnull.pkt_too_small,stats.decoder.ltnull.unsupported_type,stats.decoder.dce.pkt_too_small,stats.flow.memcap,stats.flow.tcp,stats.flow.udp,stats.flow.icmpv4,stats.flow.icmpv6,stats.flow.spare,stats.flow.emerg_mode_entered,stats.flow.emerg_mode_over,stats.flow.tcp_reuse,stats.flow.memuse,stats.defrag.ipv4.fragments,stats.defrag.ipv4.reassembled,stats.defrag.ipv4.timeouts,stats.defrag.ipv6.fragments,stats.defrag.ipv6.reassembled,stats.defrag.ipv6.timeouts,stats.defrag.max_frag_hits,stats.tcp.sessions,stats.tcp.ssn_memcap_drop,stats.tcp.pseudo,stats.tcp.pseudo_failed,stats.tcp.invalid_checksum,stats.tcp.no_flow,stats.tcp.syn,stats.tcp.synack,stats.tcp.rst,stats.tcp.segment_memcap_drop,stats.tcp.stream_depth_reached,stats.tcp.reassembly_gap,stats.tcp.overlap,stats.tcp.overlap_diff_data,stats.tcp.insert_data_normal_fail,stats.tcp.insert_data_overlap_fail,stats.tcp.insert_list_fail,stats.tcp.memuse,stats.tcp.reassembly_memuse,stats.detect.alert,stats.app_layer.flow.http,stats.app_layer.flow.ftp,stats.app_layer.flow.smtp,stats.app_layer.flow.tls,stats.app_layer.flow.ssh,stats.app_layer.flow.imap,stats.app_layer.flow.msn,stats.app_layer.flow.smb,stats.app_layer.flow.dcerpc_tcp,stats.app_layer.flow.dns_tcp,stats.app_layer.flow.failed_tcp,stats.app_layer.flow.dcerpc_udp,stats.app_layer.flow.dns_udp,stats.app_layer.flow.failed_udp,stats.app_layer.tx.http,stats.app_layer.tx.ftp,stats.app_layer.tx.smtp,stats.app_layer.tx.tls,stats.app_layer.tx.ssh,stats.app_layer.tx.smb,stats.app_layer.tx.dcerpc_tcp,stats.app_layer.tx.dns_tcp,stats.app_layer.tx.dns_udp,stats.flow_mgr.closed_pruned,stats.flow_mgr.new_pruned,stats.flow_mgr.est_pruned,stats.flow_mgr.bypassed_pruned,stats.flow_mgr.flows_checked,stats.flow_mgr.flows_notimeout,stats.flow_mgr.flows_timeout,stats.flow_mgr.flows_timeout_inuse,stats.flow_mgr.flows_removed,stats.flow_mgr.rows_checked,stats.flow_mgr.rows_skipped,stats.flow_mgr.rows_empty,stats.flow_mgr.rows_busy,stats.flow_mgr.rows_maxlen,stats.file_store.open_files,stats.dns.memuse,stats.dns.memcap_state,stats.dns.memcap_global,stats.http.memuse,stats.http.memcap,tcp.ecn,tcp.cwr
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863

In [8]:
data4 = data3

In [9]:
data4 = data4.rename(columns ={"dns.rrname": "domain_tld"})

In [10]:
pd.set_option('display.max_rows', None)
data4 = data4.filter(['domain_tld'])
data4


,domain_tld
2,api.wunderground.com
38,stork79.dropbox.com
39,hpca-tier2.office.aol.com.ad.aol.aoltw.net
44,safebrowsing.clients.google.com.home
48,fxfeeds.mozilla.com
53,www.metasploit.com
56,aolmtcmxm03.office.aol.com
64,aolmtcmxm02.office.aol.com.ad.aol.aoltw.net
94,aolmtcmxm02.office.aol.com
100,hpca-tier2.office.aol.com


10. Del dataframe de dominios únicos de tipo A, obtenga el TLD (top level domain) utilizando Flare
como una columna nueva llamada domain_tld, y elimine todas las demás columnas.

In [11]:
domain = []
for col, row in data4.iterrows():
    domain.append(domain_tld_extract(row['domain_tld']))

data4['domain_tld'] = domain

11. Busque y elimine un dominio que es un carácter vacío ‘’. También elimine los siguientes
dominios:


In [12]:
remove = ['', ' ', '192.168.22.110phpmyadmin', '192.168.22.110phpmyadmin.localdomain', '"192.168.206.56"', '192.168.26-27.0', '192.168.21.1201', '1922.168.22.254', '1922.168.22.254.home', '192.168.21-28.0.home', '192.168.22.201:', '192.168.22.201:.stayonline.net']
clean = data4[~data4['domain_tld'].isin(remove)]

In [13]:
clean = clean.drop_duplicates(subset=['domain_tld'])

12. Obtenga los TLD únicos, debe obtener 102 registros


In [14]:
len(clean)

102

Parte 2

13. Utilice el clasificador proporcionado, debe pasarle como parámetro el dataframe con la
columna domain_tld, y asignar el resultado a un nuevo DF.

In [15]:
import clasificador

In [16]:
df_clasificador = clasificador.clasificacion(clean)

c:\Users\gabri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [17]:
df_clasificador

,domain_tld,isDGA
2,wunderground.com,0
38,dropbox.com,1
39,aoltw.net,1
44,safebrowsing.clients.google.com.home,0
48,mozilla.com,0
53,metasploit.com,1
56,aol.com,0
111,google.com,0
133,wpad.home,0
251,stayonline.net,0


14. Filtre aquellos considerados como DGA (valor 1 ) y muéstrelos. Recuerde que los modelos de
ML ofrecen una predicción, pero los resultados pueden ser falsos positivos y falsos negativos,
por lo que no podemos fiarnos por completo de esta clasificación y debemos seguir
indagando. Ahora tenemos solo 35 dominios sospechosos!

In [18]:
df_DGA = df_clasificador[(df_clasificador['isDGA'] == 1)]

In [19]:
print("DGA: ", len(df_DGA))

DGA:  35


Parte 3

15. Ahora ya tenemos un listado de dominios reducido y considerado como sospechoso, por lo
que debemos aplicar dominio experto para encontrar los verdaderos registros malicioso.Utilizando la lista de Cisco Umbrella incluido en Flare, indique si los dominios encontrados por
el clasificador DGA de la parte 2 se encuentran en este top 1000000. Deberíamos haber
reducido los registros a 12!

In [20]:
umbrella = pd.read_csv("./flare/data/umbrella/top-1m.csv")

In [21]:
umbrella.rename(columns={'netflix.com': 'domain_tld'}, inplace=True)


In [22]:
df_DGA['matches'] = df_DGA['domain_tld'].isin(umbrella['domain_tld'])


<ipython-input-22-2c112a0f180d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_DGA['matches'] = df_DGA['domain_tld'].isin(umbrella['domain_tld'])


In [26]:
data5 = df_DGA[df_DGA['matches'] == True]

data5


,domain_tld,isDGA,matches
38,dropbox.com,1,True
53,metasploit.com,1,True
3266,windows.com,1,True
8104,backtrack-linux.org,1,True
13241,phpmyadmin.net,1,True
23995,microsoft.com,1,True
36126,postgresql.org,1,True
43948,freepbx.org,1,True
58278,flickr.com,1,True
74253,facebook.com,1,True


16. Si son considerados por el clasificador como DGA, y no están en el top de Cisco Umbrella es
muy probable que sean maliciosos. Para confirmar esto podemos buscar la fecha de creación
del dominio. Cree una función qué en base al dominio, devuelva la fecha de creación de este
(utilice la librería whois para esto).

In [27]:
test = []
virus = []

for value in data5['domain_tld']:
    creationDate = wh.whois(value).creation_date
    
    if type(creationDate) is list:
        test.append((creationDate[0].strftime('%m/%d/%Y')))
        virus.append(value)

    elif creationDate is None:
        test.append('None')
        virus.append(value)

    else:
        test.append((creationDate).strftime('%m/%d/%Y'))


data5['creation_date'] = test

<ipython-input-27-e3ba3162a263>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data5['creation_date'] = test


17. Muestre la fecha de creación para cada uno de los dominios clasificados como DGA. ¿Cuáles
son los dominios que podemos confirmar como sospechosos? (Deben ser cinco dominios)

In [28]:
data5

,domain_tld,isDGA,matches,creation_date
38,dropbox.com,1,True,06/28/1995
53,metasploit.com,1,True,06/10/2003
3266,windows.com,1,True,09/11/1995
8104,backtrack-linux.org,1,True,04/29/2009
13241,phpmyadmin.net,1,True,04/02/2002
23995,microsoft.com,1,True,05/02/1991
36126,postgresql.org,1,True,10/22/1996
43948,freepbx.org,1,True,10/11/2005
58278,flickr.com,1,True,11/22/2003
74253,facebook.com,1,True,03/29/1997


18. Los dominios DGA son conocidos por formarse de forma aleatoria: secuencias aleatorias de
caracteres, no palabras. Indique que dominios sospechosos tienen este patrón y que pueden
confirmarse como dominios DGA (deben ser dos).

In [29]:
virus = data5[(data5['creation_date'] == 'None')]
virus

,domain_tld,isDGA,matches,creation_date
119201,clarkson.edu,1,True,None
119226,wisc.edu,1,True,None
